## Transformers로 뭘 할 수 있을까요?



Transformers와 Datasets 라이브러리를 설치해줍니다 (-q 옵션은 로그를 생략하기 위해 사용했습니다)

In [1]:
!pip install transformers[sentencepiece] datasets -q

pipeline api는 inference 과정을 위해 전처리등 여러가지 과정을 압축한 high-level api입니다. 아래와 같이 사용할 수 있습니다.

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

리스트형 입력을 사용하면 한문장뿐만 아니라 여러문장도 처리할 수 있습니다.

In [3]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

이 파이프라인은 프리트레이닝된 모델을 감성분석 데이터셋에 파인튜닝한 모델을 사용했습니다. 모델은 classifier 객체를 생성할때 다운로드 및 캐싱되게 됩니다. 그러므로 다음번 객체를 생성할땐 추가로 다운받지 않아도 됩니다.

pipeline에 텍스트를 입력으로 넣게되면 다음과 같이 크게 3가지 과정을 거치게 됩니다.

1. 텍스트가 모델이 이해할 수 있는 포멧으로 전처리 됩니다
2. 전처리된 값이 모델에 입력됩니다.
3. 모델의 예측값이 인간이 이해할 수 있는 형태로 후처리됩니다.


사용 가능한 형태의 pipeline은 [아래](https://huggingface.co/docs/transformers/main_classes/pipelines)와 같이 여러 종류로 이루어져있습니다.

- feature-extraction (get the vector representation of a text)
- fill-mask
- ner (named entity recognition)
- question-answering
- sentiment-analysis
- summarization
- text-generation
- translation
- zero-shot-classification

이제부터 각 pipeline을 한번 살펴보겠습니다.


## Zero-shot classification
조금은 도전적인 태스크부터 다뤄보겠습니다. 보통의 텍스트 분류 문제는 레이블 데이터를 모으고, 분류기를 학습해서 문제를 해결합니다. 하지만 이는 시간이 드는 문제고, 도메인 전문가의 도움이 필요할 수도 있습니다. 이러한 경우 `zero-shot-classification` pipeline은 매우 유용하게 쓰일 수 있습니다. zero-shot은 학습을 않고 바로 분류하는 경우를 말하는데요. 분류할 문장과 다양한 클래스를 모델에 입력으로 넣어주면, 문장과 클래스의 연관도를 확률로 출력해줍니다. 내부적으로는 `MNLI`데이터셋으로 파인튜닝된 모델을 사용합니다.


In [4]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


{'labels': ['education', 'business', 'politics'],
 'scores': [0.8445989489555359, 0.11197411268949509, 0.04342697188258171],
 'sequence': 'This is a course about the Transformers library'}

위와 같이 분류하고자하는 도메인 데이터에 대해서 파인튜닝할 필요가 없기 때문에 *zero-shot* 이라고 부릅니다


## Text generation
지금부터는 텍스트 생성을 위한 pipleline을 살펴보겠습니다. 텍스트 생성을 위한 메인 아이디어는 프롬프트(prompt; 생성될 문장 앞에 들어가는 텍스트)를 제공하는 것입니다. 프롬프트가 입력되면 모델은 남은 영역을 문장 생성을 통해 완성시키게됩니다.   
텍스트 생성은 기본적으로 랜덤 속성이 있기 때문에 아래와 같은 문장이 생성되지 않더라도 정상입니다.

In [5]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to",
          max_length=30,
          num_return_sequences=2,
          )

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "In this course, we will teach you how to construct a user-friendly app using your existing browser experience. We'll also create custom content and design"},
 {'generated_text': 'In this course, we will teach you how to use the Mapper to learn more. We will have a look at some more basics of Mapper'}]

`num_return_sequences`와 `max_length` 옵션을 통해 생성할 문장 개수 및 문장 길이를 조절할 수 있습니다.

## Model Hub에 있는 모델로 pipeline 만들기
위에서 나온 예제외에 다른 모델을 사용해보고 싶다면 허깅페이스의 Model hub (https://huggingface.co/models)에 있는 다른 모델을 이용해볼 수 있습니다.
다른 text-generation 모델은 [이곳](https://huggingface.co/models?pipeline_tag=text-generation)에서 확인할 수 있습니다. 


이번에는 [distilgpt2](https://huggingface.co/distilgpt2) 모델을 사용해보겠습니다.


In [6]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to best follow the rules of the Bitcoin Blockchain.'},
 {'generated_text': "In this course, we will teach you how to perform these basic rules to prepare for an amazing experience. We don't have the usual rules to prepare"}]

[모델 허브](https://huggingface.co/models)에서 필터를 사용하면, 원하는 Tasks와 Languages에 맞는 모델을 찾을 수 있습니다. (한국어는 `ko` 태그를 통해 찾을 수 있습니다)

![model_filter](https://user-images.githubusercontent.com/7252598/145123053-ffea71a8-a22e-4c17-b4f0-3f28fb6b4be0.gif)


## Inference API
모든 모델은 허깅페이스에서 제공하는 Inference API를 통해 웹브라우저에서 테스트해볼 수 있습니다.

![inference api](https://user-images.githubusercontent.com/7252598/145122531-3e523309-e038-4bdd-8e18-4d48174b6fe6.gif)


## Mask filling
이번에 소개할 파이프라인은 `fill-mask`입니다. 쉽게말하면 빈칸 채우기 task입니다. 빈칸이 있는 텍스트가 주어졌을때 확률적으로 가장 적절한 단어를 채워줍니다. 빈칸은 특수 토큰인 `<mask>`로 표기하고, `top_k` 인자는 확률이 높은 단어를 몇개까지 출력할지를 결정합니다.

In [7]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


[{'score': 0.196198508143425,
  'sequence': 'This course will teach you all about mathematical models.',
  'token': 30412,
  'token_str': ' mathematical'},
 {'score': 0.040527332574129105,
  'sequence': 'This course will teach you all about computational models.',
  'token': 38163,
  'token_str': ' computational'}]

## Named entity recognition
개체명인식은 입력 텍스트에서 person(PER), location(LOC), organization(ORG)등에 대응되는 부분(토큰)을 찾아내는 태스크입니다.

In [8]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:129: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


[{'end': 18,
  'entity_group': 'PER',
  'score': 0.9981694,
  'start': 11,
  'word': 'Sylvain'},
 {'end': 45,
  'entity_group': 'ORG',
  'score': 0.97960186,
  'start': 33,
  'word': 'Hugging Face'},
 {'end': 57,
  'entity_group': 'LOC',
  'score': 0.99321055,
  'start': 49,
  'word': 'Brooklyn'}]

transformer 계열의 모델은 입력형태가 서브워드형태로 이루어지기 때문에 실제 엔티티는 서브워드토큰 마다 태깅되게 됩니다. `grouped_entities=True` 조건은 이러한 서브워드들을 하나의 단어 형태로 변환해줍니다.
`grouped_entities=False`로 할 경우 `Hugging Face`-> `Hu ##gging Face` 각각에 `ORG` 클래스가 태깅되게 됩니다.

## Question answering
question answering은 질문과 컨텍스트를 입력으로 넣었을때, 컨텍스트내에서 질문의 답과 가장 연관이 많은 부분을 `extractive`하게 추출하는 태스크입니다.

In [9]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


{'answer': 'Hugging Face', 'end': 45, 'score': 0.6949771046638489, 'start': 33}

## Summarization
요약은 주어진 본문중 중요한 부분 위주로 글을 간추리는 태스크입니다. 텍스트 생성과 같이 `max_length`, `min_length` 인자로 글의 양을 조절할 수 있습니다.

In [10]:
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]

## Translation
간단하게 번역할 수 있는 파이프라인도 제공됩니다. model hub에서 번역할 언어쌍이 명시된 모델([Helsinki-NLP/opus-mt-ko-en](https://huggingface.co/Helsinki-NLP/opus-mt-ko-en))을 선택해서 사용할 수 있습니다. 텍스트 생성과 같이 `max_length`, `min_length` 인자로 글의 양을 조절할 수 있습니다.

In [11]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ko-en")
translator("Huggingface로 배우는 NLP는 멋져")

[{'translation_text': 'The NLP that you learn with Huggingface is great.'}]